Таблица **video_products** хранит данные о кинолентах разного типа; а как выяснить, сколько в таблице фильмов каждого типа?

Можно сделать так:
1. Через `DISTINCT` получить список типов из колонки `product_type`:

    `SELECT DISTINCT product_type FROM video_products;`

    Вернётся четыре типа: «Мультсериал», «Мультфильм», «Сериал», «Фильм».

2. Отправить запрос, подсчитывающий количество записей с определённым типом

    `SELECT product_type, COUNT(*) FROM video_products WHERE product_type='Мультсериал';`

    Вернётся ответ: `('Мультсериал', 2)`.

3. Повторить этот запрос для всех типов фильмов — и сводка готова.
Получается довольно неудобно и громоздко. 

***
## Группировка данных: GROUP BY

В SQL есть возможность сгруппировать записи, у которых совпадает значение определённого поля; после группировки можно проводить над полученными группами необходимые операции.

![alt text](https://pictures.s3.yandex.net/resources/S2_137_1_1682533335.png)

Самый простой вариант использования **GROUP BY** — найти «группы», объединённые одинаковым значением в заданной колонке.

```sql
SELECT product_type
FROM video_products
GROUP BY product_type;
```

Все записи с одинаковыми значениями в поле `product_type` были объединены в группы, и в ответ на запрос вернулось по одной записи из каждой группы.

```bash
('Мультсериал',)
('Мультфильм',)
('Сериал',)
('Фильм',)
```

![alt text](https://pictures.s3.yandex.net/resources/S02_131_0_1678064271.png)

После того как записи сгруппированы — к каждой из групп можно применить агрегирующие функции (**COUNT**, **MIN**, **MAX**, **AVG** или **SUM**) — и начнутся чудеса.

***
## Чудеса в примерах

**Посчитаем (COUNT), сколько в таблице фильмов каждого типа** — сколько фильмов, сколько сериалов, сколько мультфильмов и мультсериалов:

```sql
-- Показать значение поля product_type и значение счётчика...
SELECT product_type,
       COUNT(*)
FROM video_products
-- ...для групп записей с одинаковым значением в поле product_type
GROUP BY product_type;
```

```py
('Мультсериал', 2)
('Мультфильм', 1)
('Сериал', 3)    
('Фильм', 4)
```

В выборке отображаются значения полей первой записи из каждой группы. Это можно увидеть, если запросить несколько полей:

```sql
SELECT title, 
       release_year,
       product_type,
       COUNT(*)
FROM video_products
GROUP BY product_type;
```

Вернутся четыре записи — по одной из каждой группы:

```py
('Весёлые мелодии', 1930, 'Мультсериал', 2)
('Розовая пантера: Контроль за вредителями', 1969, 'Мультфильм', 1)
('Она написала убийство', 1984, 'Сериал', 3)
('Кто подставил кролика Роджера', 1988, 'Фильм', 4)
```


Продолжим эксперименты: найдём самый старый фильм (MIN) в каждой группе: 

```sql
SELECT product_type,
       MIN(release_year)
FROM video_products
GROUP BY product_type;
```

На русский язык этот запрос можно перевести так: «в таблице video_products объедини в группы записи с одинаковым полем `product_type`; покажи значение `product_type` и минимальное значение `release_year` для каждой из групп».

По результату этого запроса будет сформирована выборка:

```bash
('Мультсериал', 1930)
('Мультфильм', 1969)
('Сериал', 1984)
('Фильм', 1967)
```

Просуммируем кассовые сборы (SUM) для фильмов разного типа: 

```sql
SELECT product_type,
       SUM(gross)
FROM video_products
GROUP BY product_type; 
```

Вернётся такой ответ:

```bash
('Мультсериал', None)
('Мультфильм', None)
('Сериал', None)
('Фильм', 318868922) 
```

Python преобразовал `NULL` в аналогичное по смыслу значение `None`.

Данные, которые попадут в группировку, можно предварительно отфильтровать через `WHERE`. Объединим фильмы в группы по типам и просуммируем кассовые сборы для каждой группы; но в группы включим лишь фильмы, выпущенные после 1990-го года; выведем название группы и сумму сборов — `SUM(gross)`:

```sql
SELECT product_type,
       SUM(gross)
FROM video_products
WHERE release_year > 1990
GROUP BY product_type; 
```

Результат запроса:

```bash
('Сериал', None)
('Фильм', 137298489) 
```

Заодно выяснили, что в нашей базе нет ни мультфильмов, ни мультсериалов, выпущенных после 1990-го. 

Инструкция `GROUP BY` воспринимает ячейки с `NULL` как равные, то есть при группировке все строки с `NULL` попадут в одну группу.

Сгруппируем записи по одинаковым значениям в поле `gross` и выведем количество записей в каждой группе:

```sql
SELECT gross,
       COUNT(*)
FROM video_products
GROUP BY gross; 
```

В результате получим четыре группы:

```bash
(None, 7)
(25118063, 1)
(137298489, 1)
(156452370, 1) 
```

***
## Фильтрация групп: HAVING

Инструкция `HAVING` позволяет выполнить фильтрацию групп: она определяет, какие группы будут включены в результирующую выборку. Работа `HAVING` во многом аналогична применению `WHERE`; но `WHERE` применяется для фильтрации строк, а `HAVING` — для фильтрации групп.

Исключим из выборки все группы, в которых сумма кассовых сборов равна `NULL`:

```sql
SELECT product_type,
       SUM(gross)
FROM video_products
GROUP BY product_type
-- Это условие относится к группам:
HAVING SUM(gross) IS NOT NULL;
-- ('Фильм', 318868922) 
```

Вот небольшая шпаргалка со списком самых востребованных инструкций и ключевых слов, применяемых в SQL-запросах. 

Порядок инструкций дан именно в том порядке, в каком они должны следовать в SQL-запросе. Пишите операторы в правильном порядке (а в неправильном — не пишите!).

* `SELECT <столбцы>` — обязательно. Можно применять агрегатные функции `COUNT`, `MIN`, `MAX`, `AVG` и `SUM` (необязательно) и ключевое слово `DISTINCT` (необязательно).

* `FROM <таблица>` — обязательно.

* `WHERE <условие/фильтрация на уровне записей>` — необязательно.

* `GROUP BY <столбец (один или несколько), по которому группируются записи>` — необязательно.

* `HAVING <условие/фильтрация на уровне сгруппированных записей>` — необязательно.

* `ORDER BY <столбец (один или несколько), по которому сортируется вывод>` — необязательно.
`LIMIT <сколько записей показывать>` — необязательно.
`OFFSET <сколько записей в выборке пропустить>` — необязательно.

In [ ]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

# Напишите SQL запрос в строке.
results = cur.execute('''
SELECT category,
       AVG(price)
FROM ice_cream
GROUP BY category
''')

for result in results:
    print(result)

con.close()

Если потребуется повнимательнее рассмотреть структуру таблицы — выполните запрос `PRAGMA table_info(ice_cream);`.